<h3>Build up the network</h3>
Your Genesis cloud API token needs to be set in the environment variable GC_API_TOKEN, the name of your SSH key should be set in the variable GC_SSH_KEY_NAME. If you are sure that you won't accidently push them to github, you can also overwrite them in here.

In [1]:
import os
from create_nodes import *

In [2]:
my_client = Client(api_token_arg)
if not my_client.connect():
    exit()

gateway = create_gateway(my_client, ssh_key_arg)
create_workers(my_client, ssh_key_arg, gateway.private_ip)

Creating instance 29f931af-7689-493c-8d21-e5eef8b5a25c
instance is starting ...
public ip is: 194.61.20.84
Using image snapshot nvidia+docker for installation.


<h3>Import all the required pysyft</h3>

In [1]:
import torch
import syft as sy
from syft.grid.public_grid import PublicGridNetwork
from syft.workers.node_client import NodeClient
hook = sy.TorchHook(torch)
sy.hook.local_worker.is_client_worker = False

In [4]:
public_grid = PublicGridNetwork(hook, "http://"+gateway_instance['ip']+":5000")
brutus = NodeClient(hook,"http://"+gateway_instance['ip']+":3000")
alice = NodeClient(hook,"http://"+nodes[0].ip+":3000")

In [7]:
a = torch.tensor([1,2,3])
a_ptr = a.send(alice)
a_ptr = a.send(brutus)
a_ptr.get()

tensor([1, 2, 3])